In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08756709098815918
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 11.14912500516532
200000 21.770304718364578
300000 19.273431078090077
400000 20.759130773433192
500000 18.86830138957324
600000 21.56821180392243
700000 20.345488200494483
800000 21.938469437020075
900000 19.21791482888469
1000000 21.37738093198094
1100000 20.61161570645782
1200000 23.263902239312767
1300000 19.43700472377214
1400000 4.8754527773530265
1500000 18.871206615164624
1600000 21.249939500279318
1700000 20.5637594885331
1800000 18.225221282156575
1900000 22.124654111805523
2000000 19.110072455228085
2100000 24.29411541046453
2200000 22.83408465904791
2300000 26.861079810653226
2400000 33.08782827855016
2500000 42.879384859673976
2600000 43.553898129628074
2700000 22.45118902643854
2800000 32.503363234677174
2900000 54.957005664041716
3000000 58.88

29700000 38.485873445670904
29800000 13.110926390484376
29900000 36.10204018981804
30000000 38.527782227804686
30100000 38.08660737846509
30200000 41.21148488916457
30300000 35.47398091301648
30400000 41.42452272862514
30500000 40.3439218547361
30600000 37.449734955734556
30700000 39.90859267459799
30800000 36.40090695186428
30900000 16.186308388874387
31000000 26.467187690676028
31100000 38.289629199290204
31200000 37.79624428109746
31300000 37.1127987952597
31400000 39.73147987134603
31500000 37.44344542503388
31600000 34.63798417163032
31700000 40.48435553984588
31800000 38.00905826985298
31900000 37.463771049399845
32000000 26.992680228686034
32100000 26.20866780997579
32200000 38.02917886640824
32300000 40.08144022987648
32400000 35.69982925801751
32500000 16.496017694833892
32600000 32.037911900806705
32700000 40.01944593787018
32800000 36.63229273575322
32900000 39.43641039705006
33000000 36.73238521740968
33100000 35.807001689162576
33200000 37.691613843518454
33300000 37.37846

59700000 34.60479821295574
59800000 34.46064608753504
59900000 31.683309940306557
60000000 32.01728367982269
60100000 33.52508741822471
60200000 29.534791493380038
60300000 32.393050528968516
60400000 24.563959470487386
60500000 13.207337685924191
60600000 29.7110002442315
60700000 32.39940017777999
60800000 29.031234956790993
60900000 33.33416890907913
61000000 30.527851266248963
61100000 33.540933985966255
61200000 32.77629865814926
61300000 30.75992144216296
61400000 15.538363637395543
61500000 33.67249624273918
61600000 33.41917568594294
61700000 32.92461011399991
61800000 32.46203592355056
61900000 35.09074709661025
62000000 32.070374438311134
62100000 34.992584879103696
62200000 29.651248048819195
62300000 15.739787408811718
62400000 32.48070757309262
62500000 31.163272839882225
62600000 32.44795908020968
62700000 32.28685792698277
62800000 31.20882301811895
62900000 30.21673343352043
63000000 28.059161009352966
63100000 21.739297124174247
63200000 16.62975467721987
63300000 27.4

89600000 28.451096324608404
89700000 26.304825615021134
89800000 27.796583818503
89900000 25.940662043108315
90000000 10.569965063589756
90100000 23.647944697500005
90200000 27.641648498800144
90300000 25.832182579041994
90400000 27.561847909418088
90500000 25.68139411493411
90600000 27.65936718994229
90700000 10.706816459956553
90800000 28.66481827656635
90900000 27.802338709875652
91000000 29.831669038933903
91100000 30.105858229330654
91200000 28.39647265063163
91300000 32.205359889985694
91400000 32.90225609274766
91500000 30.149508354733968
91600000 30.5977397626278
91700000 31.67369799813247
91800000 28.53865739470883
91900000 31.10605249470654
92000000 13.685007274599494
92100000 24.363095942589673
92200000 26.86070124254878
92300000 29.26800028701833
92400000 25.93004823416053
92500000 28.51581514625442
92600000 29.08153914958958
92700000 4.584784273780433
92800000 27.849697886452944
92900000 28.609060004159474
93000000 26.06955272702421
93100000 28.196343125192513
93200000 27.

118700000 25.73408806423901
118800000 25.523648299633727
118900000 23.905720842132638
119000000 22.2125571610848
119100000 11.029093351390916
119200000 24.68223586181315
119300000 23.01397418283325
119400000 25.04719400131157
119500000 16.982003764446535
119600000 14.84356921729492
119700000 23.164260192900993
119800000 24.969873932779144
119900000 23.315506564842245
120000000 16.644502475672088
120100000 17.865521443425674
120200000 25.273601440460567
120300000 23.72019867113059
120400000 26.034485659554797
120500000 24.305110378151205
120600000 27.304927351647624
120700000 28.0779557155062
120800000 25.51315712747783
120900000 27.47296169145438
121000000 27.31151088673166
121100000 25.97555026277403
121200000 27.092978311287524
121300000 24.995607738490673
121400000 27.07592282658807
121500000 25.449142825732938
121600000 23.173482977549565
121700000 22.886851002647198
121800000 9.704310441375302
121900000 24.25543459289366
122000000 24.37656592830604
122100000 23.862382044373476
122

...

233400000 14.038940650869442
233500000 15.30998966079175
233600000 15.007681288290627
233700000 13.794557596764879
233800000 15.100381831801366
233900000 13.226561080767967
234000000 15.26941397886259
234100000 13.316486141145615
234200000 13.443192292548543
234300000 13.067606607070413
234400000 12.831141468467186
234500000 13.800895095178436
234600000 12.960232741469943
234700000 5.454702131786256
234800000 13.436028262118743
234900000 12.47938567380374
235000000 6.744416272074543
235100000 9.0568413140232
235200000 13.97736390678344
235300000 11.656823255281413
235400000 5.15866593978611
235500000 12.519688308496544
235600000 13.806471102397676
235700000 12.298141265849697
235800000 13.802981235984596
235900000 12.89129187272304
236000000 13.834618437358646
236100000 15.059918514211397
236200000 13.898033729198053
236300000 15.405075981927745
236400000 14.068060821376513
236500000 14.552306995031973
236600000 14.647101939378041
236700000 14.597470135025997
236800000 14.0157343665529

262000000 12.086640629456797
262100000 10.010710088280838
262200000 11.49132594937999
262300000 10.08754744736084
262400000 6.66159830996598
262500000 7.628465334146237
262600000 11.673180226466982
262700000 10.312301972053616
262800000 12.3129480326002
262900000 11.969266302251231
263000000 11.682186825937256
263100000 11.505530639160924
263200000 12.817817896989203
263300000 10.80519816057566
263400000 13.168594380873518
263500000 11.181707467981717
263600000 12.408561151464742
263700000 11.462940857937278
263800000 11.421163551968835
263900000 10.865624103676826
264000000 5.356939902703195
264100000 10.000910594607923
264200000 10.580068061988323
264300000 11.084880699755072
264400000 10.671565567390534
264500000 9.90785278780116
264600000 3.2448384091100353
264700000 11.68890654591514
264800000 10.447755125991012
264900000 11.228265428002935
265000000 10.803391492619614
265100000 6.456233715350658
265200000 7.118664188491742
265300000 12.324799310236829
265400000 11.482565213599653

291000000 8.622312570342336
291100000 7.612176290879077
291200000 8.720823146834663
291300000 7.748921403484154
291400000 4.383179581285196
291500000 6.100424954313546
291600000 8.622004132754249
291700000 7.892278387702264
291800000 8.419817970652835
291900000 7.75856313404444
292000000 8.927409258531798
292100000 8.468482754982844
292200000 2.4958320724624
292300000 8.795397524489244
292400000 9.485305124004793
292500000 8.92536382620951
292600000 9.513215817766502
292700000 9.374227290336421
292800000 8.836877576232768
292900000 9.477781986386281
293000000 9.128364925932942
293100000 9.26641731389714
293200000 8.671061627635835
293300000 5.888359049570996
293400000 5.669128149357143
293500000 8.748991602934638
293600000 8.006219741618176
293700000 8.595034434812774
293800000 7.507946368810792
293900000 8.389225809359795
294000000 7.4851293791922835
294100000 3.28675843390454
294200000 6.73166135243007
294300000 8.514440155994635
294400000 7.633535266269668
294500000 8.97117513579655

320300000 5.685188412477869
320400000 5.9817971637734315
320500000 5.408510532660533
320600000 5.907813344658015
320700000 5.209983393325173
320800000 5.402402002125028
320900000 4.6908291333414915
321000000 1.849864203620824
321100000 5.3262589047482924
321200000 5.275381602955932
321300000 5.6076118641197485
321400000 5.851149313002892
321500000 5.8206688925347585
321600000 5.633266049299616
321700000 6.176392101661759
321800000 5.7651386432254395
321900000 3.7682177602354843
322000000 3.7765057489058083
322100000 5.999882264559853
322200000 5.495165211957285
322300000 6.045942128851937
322400000 5.501975180378958
322500000 5.920832919985908
322600000 4.848860296185217
322700000 2.4756071836931053
322800000 5.388077888151151
322900000 5.992405331348468
323000000 5.319669807649445
323100000 5.582312092114362
323200000 5.08124203448282
323300000 5.559811927106034
323400000 4.830586732785595
323500000 5.154359002630828
323600000 3.6909362713077902
323700000 2.495358348075546
323800000 4

349300000 1.8078501969257588
349400000 2.060035702510687
349500000 2.8627513231352975
349600000 2.891721191597894
349700000 2.9029222176945724
349800000 2.941875433339356
349900000 2.7197807478506157
350000000 2.8674087350780604
350100000 2.941414135551871
350200000 2.788317773833931
350300000 2.753669593333179
350400000 2.5517016007187276
350500000 0.9492412660908977
350600000 2.265013238551201
350700000 2.85099086148009
350800000 2.627269012297185
350900000 2.9070070626068114
351000000 2.604933611458811
351100000 2.7704338051576567
351200000 2.5974501807323573
351300000 2.907203527019583
351400000 2.58950629227664
351500000 2.918713332114122
351600000 2.852262122488396
351700000 0.7097814933159484
351800000 2.795194070100975
351900000 2.6882909771758987
352000000 0.9314989704612261
352100000 2.718436932475072
352200000 2.6757801406930217
352300000 2.6033828231414944
352400000 2.6379127801560385
352500000 2.59679039001184
352600000 2.412182230040442
352700000 2.5213439886533022
352800